In [1]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/neccam/slt.git

%cd slt

!chmod +x data/download.sh
!data/download.sh

!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

!pip -q install -r requirements.txt

Mounted at /content/drive
Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.
/content/slt
--2021-01-09 13:39:48--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  47.4MB/s    in 41s     

2021-01-09 13:40:30 (44.7 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-09 13:40:30--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13

In [2]:
!cp ../drive/MyDrive/slt/early-fusion-v3/*.py signjoey/

In [3]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!unzip -q '../drive/MyDrive/slt/keypoints/train_body_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_face_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_hand_pose3d.zip' -d data

!unzip -q '../drive/MyDrive/slt/keypoints/test_keypoints.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/dev_keypoints.zip' -d data

!unzip -q ../drive/MyDrive/slt/keypoints/dev_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/train_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/test_features.zip -d data

In [4]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [5]:
body_parts = ['body_pose3d',
              'face_pose3d',
              'hand_pose3d']

for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  src_features = load_dataset_file(filename)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)
  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'
    
    new_features_list = []

    for body_part in body_parts:
      dst = os.path.join(os.path.join(wd, s), os.path.join(body_part, name))

      new_features = np.load(dst)
      new_features = new_features.reshape((new_features.shape[0], np.prod(new_features.shape[1: ])))
      new_features_list.append(new_features)
      
    
    new_features = np.concatenate(new_features_list, axis=1)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [00:45<00:00, 154.54it/s]


In [6]:
for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami2.{}".format(s)

  src_features = load_dataset_file(filename)

  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    
    new_features = np.load(dst)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [01:33<00:00, 75.60it/s]


In [7]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(3)]

list_doc['data']['feature_size'] = [1024, 417, 1024]
list_doc['training']['recognition_loss_weight'] = 40

list_doc["model"]["embeddings"] = []
list_doc["model"]["embeddings"].append(list_doc["model"]["encoder"]["embeddings"].copy())
for i in range(2):
  list_doc["model"]["encoder"]["embeddings"]["embedding_dim"] = 256
  list_doc["model"]["embeddings"].append(list_doc["model"]["encoder"]["embeddings"].copy())

list_doc["model"]["encoder"]["embeddings"]["embedding_dim"]   = 1024
list_doc["model"]["encoder"]["hidden_size"] = 1024
list_doc["model"]["decoder"]["embeddings"]["embedding_dim"]   = 1024
list_doc["model"]["decoder"]["hidden_size"] = 1024

with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
%cd slt

[Errno 2] No such file or directory: 'slt'
/content/slt


In [9]:
!python -m signjoey train configs/sign.yaml

Streaming output truncated to the last 5000 lines.
2021-01-09 14:20:21,462 ========================================================================================================================
2021-01-09 14:20:21,462 Logging Sequence: dev/14October_2009_Wednesday_tagesschau-529
2021-01-09 14:20:21,462 	Gloss Reference :	BESONDERS NORDWEST WIND STARK
2021-01-09 14:20:21,462 	Gloss Hypothesis:	********* ******** **** *****
2021-01-09 14:20:21,462 	Gloss Alignment :	D         D        D    D    
2021-01-09 14:20:21,462 	--------------------------------------------------------------------------------------------------------------------
2021-01-09 14:20:21,464 	Text Reference  :	vor allem im  nordwesten ist es *** sehr windig .
2021-01-09 14:20:21,464 	Text Hypothesis :	am  tag   ist es         ist es ist es   mal    .
2021-01-09 14:20:21,464 	Text Alignment  :	S   S     S   S                 I   S    S       
2021-01-09 14:20:21,464 ======================================================

In [10]:
!python -m signjoey test configs/sign.yaml

2021-01-09 15:47:47.706121: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-09 15:47:54.040944: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-09 15:47:54.057569: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-09 15:47:54.111250: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-09 15:47:54.111859: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-01-09 15:47:54.111915: I tensorflow/stream_executor/platform/def

In [ ]:
38.00 & 18.18 & 37.47 & 18.96